# Install Dependence Library

In [1]:
!pip install git+https://github.com/huggingface/transformers.git  # install transformer from source
# !pip install --upgrade torch datasets accelerate peft bitsandbytes trl
# !pip install --upgrade accelerate peft bitsandbytes trl
!pip install --upgrade datasets accelerate bitsandbytes  # add datasets, accelerate , bitsndbytes
!pip install langchain  langchain-community sentence-transformers chromadb  faiss-cpu #pypdf
!pip install --upgrade huggingface_hub
!pip install -qU langchain-text-splitters
# for loRA fine tuning 
!pip install --upgrade peft trl
# for advance RAG & LLM evalution 
!pip install umap-learn evaluate deepeval weave

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-aahszz20
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-aahszz20
  Resolved https://github.com/huggingface/transformers.git to commit 35b112d3442e4a9f3873191dcde7d1235b6c72eb
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 29.6 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9165401 sha256=fdf4dab1b1dc5c2f51645b6b7d36e71c1b9b71fb38d94ee1aba6f9d8b27e1817
  Stored in directory: /tmp/pip-ephem-wheel-cache-f9j34di6/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempti

# Import Library

In [2]:
import os, json, time
import gc
from IPython.display import display, Markdown

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import transformers
import torch
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM, TrainingArguments
# from langchain.document_loaders import TextLoader # old version
from langchain_community.document_loaders import TextLoader # new version
from langchain.prompts.prompt import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_community.vectorstores import FAISS, Chroma
# Text embedding / Texxt Splitter for RAG 
from langchain.text_splitter import RecursiveCharacterTextSplitter , CharacterTextSplitter # Text Splitter
from langchain.embeddings import HuggingFaceEmbeddings
from datasets import Dataset, DatasetDict, load_dataset

# Adv RAG library 


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Use Weights and Bias for LLM evaluation setup in Kaggle, click link for detail 
### use W&B need setup your Access token key
<https://www.kaggle.com/code/samuelcortinhas/weights-biases-tutorial-beginner>

In [5]:


# Enable/Disable Function
FEW_SHOT_TEST= True# False
USE_RAG = True#False #True#True
USE_WANDB = True # for  LLM evalution and debug tools  
# USE_TRAIN = True
if device.type == "cuda": #requred GPU support for fine turning 
    USE_TRAIN =  True#False#True
else:
    USE_TRAIN= False

if USE_WANDB:
    # train report to  W&B tool
    import wandb
    from kaggle_secrets import UserSecretsClient
    reportTo= "wandb"
    user_secrets = UserSecretsClient()
    my_secret = user_secrets.get_secret("wandb_api_key") 
    wandb.login(key=my_secret) # login 
else: 
    reportTo = None
    

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
device.type

'cuda'

In [7]:
if device.type == "cuda": #requred GPU support
    # for LoRA fine tuning
    from trl import SFTTrainer
    from peft import LoraConfig, PeftModel


2024-06-20 11:34:35.855726: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 11:34:35.855831: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 11:34:35.954799: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Show available project file 

In [8]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-mathematical-olympiad-prize/sample_submission.csv
/kaggle/input/ai-mathematical-olympiad-prize/AIMO Prize - Note on Language and Notation.pdf
/kaggle/input/ai-mathematical-olympiad-prize/train.csv
/kaggle/input/ai-mathematical-olympiad-prize/test.csv
/kaggle/input/ai-mathematical-olympiad-prize/aimo/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/ai-mathematical-olympiad-prize/aimo/__init__.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.safetensors.index.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00003-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/LICENSE
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00001-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/USE_POLICY.md
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer.json
/kaggle/input/llama-3/transformers/8b-c

In [9]:
sampleSubmitFile = "/kaggle/input/ai-mathematical-olympiad-prize/sample_submission.csv"
trainFile = "/kaggle/input/ai-mathematical-olympiad-prize/train.csv"
testFile = "/kaggle/input/ai-mathematical-olympiad-prize/test.csv"
mathQSATrainFile = "/kaggle/input/math-qsa-dataset/train.csv"
mathQSATestFile = "/kaggle/input/math-qsa-dataset/test.csv"
gsm8kTrainFile = "/kaggle/input/gsm8k-grade-school-math-8k-dataset-for-llm/gsm8k/main/train-00000-of-00001.parquet"
gsm8kTestFile = "/kaggle/input/gsm8k-grade-school-math-8k-dataset-for-llm/gsm8k/main/test-00000-of-00001.parquet"
mathQATrainFile = "/kaggle/input/math-qa-for-aqua-rat-dataset/MathQA/train.json"
mathQATestFile = "/kaggle/input/math-qa-for-aqua-rat-dataset/MathQA/test.json"

# EDA

In [10]:
gc.collect()

30

In [11]:
trainDF = pd.read_csv(trainFile)
trainDF

,id,problem,answer
0,229ee8,"Let $k, l > 0$ be parameters. The parabola $y ...",52
1,246d26,Each of the three-digits numbers $111$ to $999...,250
2,2fc4ad,Let the `sparkle' operation on positive intege...,702
3,430b63,What is the minimum value of $5x^2+5y^2-8xy$ w...,800
4,5277ed,There exists a unique increasing geometric seq...,211
5,739bc9,For how many positive integers $m$ does the eq...,199
6,82e2a0,Suppose that we roll four 6-sided fair dice wi...,185
7,8ee6f3,"The points $\left(x, y\right)$ satisfying $((\...",320
8,bedda4,Let $ABCD$ be a unit square. Let $P$ be the po...,480
9,d7e9c9,A function $f: \mathbb N \to \mathbb N$ satisf...,199


In [12]:
trainDF.describe()

,answer
count,10.000000
mean,339.800000
std,243.610709
min,52.000000
25%,199.000000
50%,230.500000
75%,440.000000
max,800.000000


In [13]:
trainDF.iloc[7]["answer"]

320

In [14]:
testDF = pd.read_csv(testFile)
testDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   row_id   3 non-null      int64 
 1   id       3 non-null      object
 2   problem  3 non-null      object
dtypes: int64(1), object(2)
memory usage: 200.0+ bytes


In [15]:
trainQSADF = pd.read_csv(mathQSATrainFile)
trainQSADF.head()

,problem,level,type,solution,answer
0,The United States Postal Service charges an ex...,Level 3,Prealgebra,We calculate the desired ratio for each envelo...,3
1,How many integers between 1000 and 2000 have a...,Level 4,Prealgebra,"A number with 15, 20 and 25 as factors must be...",3
2,"Given that $n$ is an integer and $0 < 4n <30$,...",Level 2,Prealgebra,"Dividing by $4$, we have $0<n<7\frac{1}{2}$. T...",28
3,How many integers between $100$ and $150$ have...,Level 4,Prealgebra,We will break up the problem into cases based ...,18
4,Regular pentagon $ABCDE$ and regular hexagon $...,Level 4,Prealgebra,We know that the sum of the degree measures of...,132


In [16]:
trainQSADF["problem"][0]

'The United States Postal Service charges an extra $\\$0.11$ in postage if the length of an envelope, in inches, divided by its height, in inches, is less than $1.3$ or greater than $2.5.$ For how many of these four envelopes must the extra $\\$0.11$ in postage be paid? \\begin{tabular}[t]{ccc}\nEnvelope & Length in inches & Height in inches\\\\\\hline\nA &6 &4\\\\\nB &9 &3\\\\\nC &6 &6\\\\\nD &11 &4\n\\end{tabular}'

In [17]:
trainQSADF.describe()

,problem,level,type,solution,answer
count,7498,7498,7498,7498,7496
unique,7498,6,7,7498,2907
top,"A function $f$ has domain $[0,2]$ and range $[...",Level 5,Algebra,"For $g(x) = 1 - f(x + 1)$ to be defined, we ne...",2
freq,1,2304,1742,1,200


In [18]:
trainGSM8KDF =pd.read_parquet(gsm8kTrainFile)
trainGSM8KDF.head()

,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<..."
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...


In [19]:
trainGSM8KDF.iloc[0]["answer"]

'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'

In [20]:
testGSM8KDF = pd.read_parquet(gsm8kTestFile)
testGSM8KDF.head()

,question,answer
0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...
1,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...
2,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...
3,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...
4,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t..."


trainMathQADF = pd.read_json(mathQATrainFile)
trainMathQADF.head()

In [21]:
trainDF["problem"][9]

'A function $f: \\mathbb N \\to \\mathbb N$ satisfies the following two conditions for all positive integers $n$:$f(f(f(n)))=8n-7$ and $f(2n)=2f(n)+1$. Calculate $f(100)$.'

In [22]:
## cleaning data set
# trainDF["problem"] = trainDF["problem"].str.replace("$", '')
# trainDF["problem"] = trainDF["problem"].str.replace("\\vert", '|')
# trainDF["problem"] = trainDF["problem"].str.replace("\\left", '')
# trainDF["problem"] = trainDF["problem"].str.replace("\\right", '')
# trainDF["problem"] = trainDF["problem"].str.replace("\\mathbb", '')

In [23]:
print(trainDF["problem"][1])

Each of the three-digits numbers $111$ to $999$ is coloured blue or yellow in such a way that the sum of any two (not necessarily different) yellow numbers is equal to a blue number. What is the maximum possible number of yellow numbers there can be?


In [24]:
print(trainDF["answer"][1])

250


# Load LLM Model

In [25]:
# define model 
USE_LLAMA3 = False # for GPU version
modelName1 = "/kaggle/input/gemma/transformers/2b-it/3"
modelName2 =  "/kaggle/input/gemma/transformers/7b-it/3" # careful memory usage , will out of Memory both CPU or GPU
modelName3 =  "/kaggle/input/llama-3/transformers/8b-chat-hf/1" 
do_sample= True 
top_p=0.95 
top_k= 2
temperature=0.2#0.7 
num_beams = 3
max_length= 512

# Quantized Config
bnb_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
)


In [26]:
device.type

'cuda'

In [27]:
if device.type == "cuda": # use 7b model gain Math performance
    if USE_LLAMA3: 
        modelSel = modelName3
        llmModel = "llama3_8b"
    else: 
        modelSel = modelName2
        llmModel = "gemma_7b"
    model = AutoModelForCausalLM.from_pretrained(modelSel , device_map="auto", quantization_config= bnb_config)   # intial with GPU quantized
    tokenizer = AutoTokenizer.from_pretrained(modelSel) # inital tokenizer
else: 
    modelSel = modelName1
    llmModel = "gemma_2b"
    model = AutoModelForCausalLM.from_pretrained(modelSel , device_map="auto")   # intial 
    tokenizer = AutoTokenizer.from_pretrained(modelSel) # inital tokenizer

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [28]:
model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear4bit(in_features=24576, out_features=3072, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
  

In [29]:
llmModel

'gemma_7b'

# Test LLM Model response with Math Question

In [30]:
def generateResponse(query, maxOutToken=256):
    """
     Direct send message to gemini, get response
    """
    inputIds = tokenizer(query, return_tensors="pt").to(device)
    response = model.generate(**inputIds , 
                              do_sample =True,
                              top_p = 0.95,
                              top_k= 2,
                              temperature= 0.2, #0.7,#0.3,#0.7,
#                               max_length=maxOutToken,
                              max_new_tokens= maxOutToken,
                             )
#     return tokenizer.decode(response[0])
    return tokenizer.decode(response[0][len(inputIds["input_ids"]):], skip_special_tokens = True)
    

In [31]:
def generateReponseInst(promptTemp, query, maxOutToken=256):
    """
    Insert prompt template instruction with message
    """
#     prompt = f"""{promptTemp}\nQuestion: {query}\nAnswer: 
#     """
    prompt = f"""{promptTemp}\nQuestion: {query}\n### Instruction: Given Answer in JSON format with key 'answer' and 'explanation' ### 
    """
    inputIds = tokenizer(prompt, return_tensors="pt").to(device)
    response = model.generate(**inputIds,
                              do_sample =True,
                              top_p = 0.95,
                              top_k= 2,
                              temperature= 0.2, #0.7, # 0.3,#0.7,
#                               max_length=maxOutToken,
                              max_new_tokens= maxOutToken,)
#     return tokenizer.decode(response[0]) # reutrn 
    return tokenizer.decode(response[0][len(inputIds["input_ids"]):], skip_special_tokens = False)

In [32]:
def generateReponseRAG(promptTemp, ragContext, query, maxOutToken=256):
    """
    Use Insert prompt insturction, RAG retrieve with query 
    """
    info = "\n".join(ragContext)
    prompt = f"""
    {promptTemp}\n
    Question: {query}\n
    Information: {info}\n
    Answer:
    """
    inputIds = tokenizer(prompt, return_tensors="pt").to(device)
    response = model.generate(**inputIds, 
                              do_sample =True,
                              top_p = 0.95,
                              top_k= 2,
                              temperature= 0.2, #0.7, #0.3, #0.7,
#                               max_length=maxOutToken,
                              max_new_tokens= maxOutToken,)
    return tokenizer.decode(response[0][len(inputIds["input_ids"]):], skip_special_tokens=True)


In [33]:
import re


# Define Parser function to extract answer

In [34]:
from json.decoder import JSONDecodeError
def isInteger(text):
    try:
        if int(text) >= 0:
            return True
        else:
            return False
    except ValueError:
        return False
    

def llmJSONParser1(txt):
#     print(txt)
    try:
        txt = txt.replace("<eos>", "")
        subTxt = txt.split("```")
        subTxt[1] = subTxt[1].replace("json", "")
        subTxt[1] = subTxt[1].replace("<eos>", "")
#             print(subTxt[1])
        jsonTxt = json.loads(subTxt[1])
    except  JSONDecodeError as e:
        print("Error LLM JSON parser", e)
        return None
    except :
        print(f"""Error LLM JSON parser input txt {txt}""" )
        return None
    return jsonTxt


def llmJSONParser2(txt):
#     print(txt)
    try:
        subText = txt.split("{")
        start = txt.find("{")
        end = txt.find("}")
        print(f"Start loc: {start}, End loc: {end}")
        subString = txt[start:end+1]
        print(subString)
        jsonTxt = json.loads(subString)
    except  JSONDecodeError as e:
        print("Error LLM JSON parser", e)
        return None
    except :
        print(f"""Error LLM JSON parser input txt {txt}""" )
        return None
    return jsonTxt
    
def llmJSONParser3(txt):
    '''
    Manual JSON answer parser without , json library 
    '''
    try:
        subText = txt.split("{") # split several {} in list
        for txtSeg in subText: # loop in list to find answer
            end = txtSeg.find("}") # find end position in text segment
            sub = txtSeg[:end] #subsring with {} context
            temp = sub.replace("*", "") # remove * symbol
            temp = temp.replace("\"", "") # reomve \" symbol
            temp = temp.lower() # convert to lower case
            answerloc = temp.find("answer:") # find key word "answer" position
            if answerloc != -1:
                print(f"find answer location : {answerloc}")
                newTxt = temp[answerloc:] # substring start answer
#                 print("Temp: ", temp)
                subTxt = newTxt.split("\n")
                #       print(subTxt)
                rel =subTxt[0][len("answer:"):].strip() # get answer value with remove space
                rel= rel.replace(',', '') # remove , symbol
                print(rel)
                if isInteger(rel):
                    return rel
                else:
                    continue # not find the value
#                 print(rel)
                
                
        return None # can't find answer

    except  JSONDecodeError as e:
        print("Error LLM JSON parser", e)
        return None
    except :
        print(f"""Error LLM JSON parser input txt {txt}""" )
        return None
    return jsonTxt
    

In [35]:
import re
def getAnswerParser1(txt):
    """
    when json parser failure, use 
    """
    # find json format first
    pattern = re.compile(r'^{')
#     pattern = re.compile(r'Response\s*:\s*(.*)')
    extracted_texts = []
    count = 0
    for text in txt:
        match = re.search(pattern, txt)
        if match:
            extracted_texts.append(match.group(1))
        else:
            extracted_texts.append("")
            count += 1
    print(f"Pattern not found for {count} results.")
    return extracted_texts

def getAnswerParser2(txt):
    """
    when json parser failure, seem answer not JSON format, 
    use 'answer' for key word search final answer 
    """
    # find answer  
    temp = txt.replace("*", "") # remove * symbol
    temp = temp.replace("\"", "") # reomve "" symbol
    temp = temp.lower() # convert to lower case
    # find answer key word
    start = temp.find("answer:")
    print(f"Start loc: {start}")
    subStr = temp[start:]
    if start != -1:
        subTxt = subStr.split("\n")
#       print(subTxt)
        rel =subTxt[0][len("answer:"):].strip() # get answer value with remove space
        rel= rel.replace(',', '') # remove , symbol
        print(rel)
#         if isInteger(rel):
        return rel
#                 print(rel)
#         return rel
    
    print(subStr)
#     return None



def getAnswer(text):
    try:
        ans = re.search(r'Answer:\s*([\s\S]+)', text).group(1).strip()
        ans = ans.replace(",","")
        if isInteger(ans):
            return int(ans) % 1000
        else:
            return 0
    except:
        return 0

In [36]:
# rel = generateResponse("What is Data Science?", maxOutToken=100)
# print(rel)

## Zero Shot For testing Gemma LLM model Math performance

In [37]:
# %%time 
# # Test without prompt instruction for assist reasoning
# result = []
# for i, qa in enumerate(trainDF["problem"][:3]):
#     rel = generateResponse(qa, maxOutToken=256)
#     print(f"Qusetion {i+1} : {qa}\nResponse : {rel}")
#     print("--"*20)
#     result.append(rel)


In [38]:
gc.collect()

51

### Compare result with Zero shot learning

In [39]:
# compare the result, 
# for i, rel in enumerate(result):
#     print(f"Question {i} Result: {rel}\n")
#     print(f"""Actual Ans: {trainDF["answer"].iloc[i]}""")
#     print("--"*20)
    

## Result from zero shot no prompt instruction: all answer is failure

In [40]:
def generateFewShotTrain(df,sample=5):
    """
    
    """
    out= ""
    for i in range(sample):
        out +=f"""Q: {df["problem"].iloc[i]}\nA:{df["answer"].iloc[i]}\n"""
    return out
    

In [41]:
fewTrainSample = generateFewShotTrain(trainDF, 3)
print(fewTrainSample)

Q: Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?
A:52
Q: Each of the three-digits numbers $111$ to $999$ is coloured blue or yellow in such a way that the sum of any two (not necessarily different) yellow numbers is equal to a blue number. What is the maximum possible number of yellow numbers there can be?
A:250
Q: Let the `sparkle' operation on positive integer $n$ consist of calculating the sum of the digits of $n$ and taking its factorial, e.g. the sparkle of 13 is $4! = 24$. A robot starts with a positive integer on a blackboard, then after each second for the rest of eternity, replaces the number on the board with its sparkle. For some `special' numbers, if they're the first number, then eventually every number that appears will be less than 6. How many such special numbers are there with at most

## Testing with Few Shot learning wihout prompt instruction

In [42]:
gc.collect()

0

In [43]:
if USE_WANDB and FEW_SHOT_TEST:
     # Start a new wandb run
    runTask1 = wandb.init(project='ai-math-solving-few-show', job_type="generation", anonymous="allow")
    # define W&B Table
    wandbCol =  ["model", "question", "label_answer", "llm_generate", "llm_answer"]
    wandbFewShotTable =wandb.Table(columns=wandbCol)

wandb: Currently logged in as: johnsonhk88. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240620_113653-2r2h3ah9
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run dauntless-feather-5
wandb: ⭐️ View project at https://wandb.ai/johnsonhk88/ai-math-solving-few-show
wandb: 🚀 View run at https://wandb.ai/johnsonhk88/ai-math-solving-few-show/runs/2r2h3ah9


In [44]:
%%time 
if FEW_SHOT_TEST:
    # Test a few shot learning for assist reasoning
    result = []
    for i, qa in enumerate(trainDF["problem"][4:6]):
        rel = generateReponseInst(fewTrainSample, qa, maxOutToken=2048)
        print(f"Qusetion {i+4} : {qa}\nResponse : {rel}")
        print("--"*20)
        result.append(rel)
#         wandbFewShotTable.add_data(fewTrainSample, llmModel, qa, ans, rel, None)


Qusetion 4 : There exists a unique increasing geometric sequence of five 2-digit positive integers. What is their sum?
Response : Q: Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?
A:52
Q: Each of the three-digits numbers $111$ to $999$ is coloured blue or yellow in such a way that the sum of any two (not necessarily different) yellow numbers is equal to a blue number. What is the maximum possible number of yellow numbers there can be?
A:250
Q: Let the `sparkle' operation on positive integer $n$ consist of calculating the sum of the digits of $n$ and taking its factorial, e.g. the sparkle of 13 is $4! = 24$. A robot starts with a positive integer on a blackboard, then after each second for the rest of eternity, replaces the number on the board with its sparkle. For some `special' numbers, if they're the

In [45]:
if FEW_SHOT_TEST:
    # compare the result, 
    for i, rel in enumerate(result):
        jsonTxt= llmJSONParser3(rel) # use llm JSON parser get answer
        if jsonTxt:
            # print(f"""Question {i} \n{trainDF["problem"].iloc[i+4]}\n\nResult: {jsonTxt["answer"]}""")
            print(f"""Question {i} \n{trainDF["problem"].iloc[i+4]}\n\nResult: {jsonTxt}""")# for llm json Parser3
            print(f"""Actual Ans: {trainDF["answer"].iloc[i+4]}""")
            if USE_WANDB:
                wandbFewShotTable.add_data( llmModel, 
                                           trainDF["problem"].iloc[i+4], 
                                           trainDF["answer"].iloc[i+4],
                                            rel, # llmm response
                                            jsonTxt)
        else:
#           ans= getAnswer(rel)
            ans = getAnswerParser2(rel)
            if ans:
                print(f"""Question {i} \n{trainDF["problem"].iloc[i+4]}\n\nResult: {ans}""")
                print(f"""Actual Ans: {trainDF["answer"].iloc[i+4]}""")
                if USE_WANDB:
                    wandbFewShotTable.add_data( llmModel, 
                                           trainDF["problem"].iloc[i+4], 
                                           trainDF["answer"].iloc[i+4],
                                            rel, # llmm response
                                            ans)
        print("--"*20)

find answer location : 9
123
Question 0 
There exists a unique increasing geometric sequence of five 2-digit positive integers. What is their sum?

Result: 123
Actual Ans: 211
----------------------------------------
find answer location : 9
12
Question 1 
For how many positive integers $m$ does the equation \[\vert \vert x-1 \vert -2 \vert=\frac{m}{100}\] have $4$ distinct solutions?

Result: 12
Actual Ans: 199
----------------------------------------


In [46]:
len(result[0])

2938

# Prompt Engineering
## Create instruction prompt

In [47]:
gc.collect()

30

In [48]:
if device.type =="cuda":
    torch.cuda.empty_cache()

In [49]:
templatePrompt1 = """
you are act as Mathematician, solve the math problem reasonable and logical from given question. 
Answer the result from question and think step-by-step to make answer. 
"""

templatePrompt2 = """
you are act as Mathematician, solve the math problem reasonable and logical from given question. 
The solving the answer and thinking step by step from question\n{question}. Only require given final result in JSON format with key "answer" and "explanation" 
"""


templatePrompt3 = """
you are act as Mathematician, solve the math problem reasonable and logical from given question. 
The solving the answer and thinking step by step from question\n{question}. Only require given final result answer, output in JSON format with key "answer" and "explanation" 
"""

templatePrompt4 = """
you are act as Mathematician, solve the math problem reasonable and logical from given question. 
The solving the answer and thinking multiple step by step from question : {question}.
Careful to logical reasoning  multiple steps to find the best solution
Only require given final answer in JSON format with key "answer" and "explanation"  
"""
    
templatePrompt5 = """
you are act as Mathematician, solve the math problem reasonable and logical from given question. 
The solving the answer and thinking multiple step by step from question : {question}.
Careful to logical reasoning  multiple steps to find the best solution
\nOnly Output answer in json format with key "answer" and "explanation" 
"""

templatePrompt15 = """
### Instruction: you are act as Mathematician, solve the math problem reasonable and logical from given question.###
The solving the answer and thinking multiple step by step from Question: {question}.
Careful to logical reasoning multiple steps to find the best solution.
Only Output answer in json format with key "answer" and "explanation".

"""

## RAG base Template Prompt

In [50]:
templatePrompt6 = """
you are act as Mathematician, solve the math problem reasonable and logical from given question follow the requirement as below:
CONTEXT: {context}
Use context for reference help to solve math question step by step.
Solving the answer and rethinking multiple step by step from Question: {question}.
Careful to logical reasoning  multiple steps to find the best solution 
Only Output answer in json format with key "answer" and "explanation" 
"""

In [51]:
templatePrompt7 = """
you are act as Mathematician, solve the math problem reasonable and logical from given question follow the requirement as below:
CONTEXT: {context}
The context contain sample question, solution and answer with similarity to user Question. Use context for assist to solve math question step by step.  
Solving the answer and rethinking multiple step by step from user Question.
User Question: {question}.
Careful to logical reasoning  multiple steps to find the best solution 
Only Output answer in json format with key "answer" and "explanation" 
"""

## Ohter Prompt Engineering with more instruction step

In [52]:
templatePrompt8 = """
You are required to answer Arithmetic Reasoning questions. Please follow the steps below as you go about solving the problem. 
1. All problems should be solved in as many steps as required and each step should be numbered.
2. All calculations should be clear and simple to understand.
3. Provide an accurate final answer at the end of your reasoning in the following format: 
4. Output result only in JSON format with key "answer" and "explanation".
Question: {question}
You'll be evaluated on:
- how easier are the calculations done to understand?
- how accurate is the answer?
Output result only in JSON format with key
"""

templatePrompt18 = """
### Instruction: You are required to answer Arithmetic Reasoning questions. Please follow the steps below as you go about solving the problem.### 
1. All problems should be solved in as many steps as required and each step should be numbered.
2. All calculations should be clear and simple to understand.
3. Provide an accurate final answer at the end of your reasoning in the following format: 
4. Output result only in JSON format with key "answer" and "explanation".
Question: {question}
You'll be evaluated on:
- how easier are the calculations done to understand?
- how accurate is the answer?
Output result only in JSON format with key
"""

In [53]:
#no.of cores each processor is having
!lscpu | grep 'Core(s) per socket'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Core(s) per socket:                 2


In [54]:
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Thread(s) per core:                 2


In [55]:
# %%time 
# # Test without prompt instruction for assist reasoning
# questionPrompt2 = PromptTemplate(input_variables=["question"], template=templatePrompt2)
# result = []
# for i, qa in enumerate(trainDF["problem"][0:3]):
#     finalPrmpt2 = questionPrompt2.format(
#         question=qa
#     )""
#     rel = generateResponse(finalPrmpt2, maxOutToken=512)
#     print(f"Qusetion {i+4} : {qa}\nResponse : {rel}")
#     print("--"*20)
#     result.append(rel)

In [56]:
%%time 
if FEW_SHOT_TEST:
    # Test without prompt instruction for assist reasoning
    newPrompt = PromptTemplate(input_variables=["question"], template=templatePrompt15)
    result = []
    for i, qa in enumerate(trainDF["problem"][0:3]):
        finalPrmpt = newPrompt.format(
            question=qa
        )
        rel = generateResponse(finalPrmpt, maxOutToken=2048)
        print(f"Qusetion {i+1} : {qa}\nResponse : {rel}")
        print("--"*20)
        result.append(rel)

Qusetion 1 : Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?
Response : 
### Instruction: you are act as Mathematician, solve the math problem reasonable and logical from given question.###
The solving the answer and thinking multiple step by step from Question: Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?.
Careful to logical reasoning multiple steps to find the best solution.
Only Output answer in json format with key "answer" and "explanation".

**Answer:**
{
  "answer": 26,
  "explanation": "Given parabola and line intersect at points A and B, distance between A and B is 6. The distance from a point to a parabola i

In [57]:
if FEW_SHOT_TEST:
    print(len(result))

3


In [58]:
# jsonTxt= llmJSONParser(result[0])
# jsonTxt
# result[0]

In [59]:
# jsonTxt["answer"]

In [60]:
if FEW_SHOT_TEST:# compare the result, 
    for i, rel in enumerate(result):
        jsonTxt= llmJSONParser3(rel) # use llm JSON parser get answer
        if jsonTxt:
            # print(f"""Question {i} \n{trainDF["problem"].iloc[i+4]}\n\nResult: {jsonTxt["answer"]}""") # for llm json parser1
            print(f"""Question {i} \n{trainDF["problem"].iloc[i]}\n\nResult: {jsonTxt}""")# for llm json Parser3
            print(f"""Actual Ans: {trainDF["answer"].iloc[i]}""")
            if USE_WANDB:
                wandbFewShotTable.add_data( llmModel, 
                                           trainDF["problem"].iloc[i], 
                                           trainDF["answer"].iloc[i],
                                            rel, # llmm response
                                            jsonTxt)

        else:
#           ans= getAnswer(rel)
            ans = getAnswerParser2(rel)
            if ans:
                print(f"""Question {i} \n{trainDF["problem"].iloc[i]}\n\nResult: {ans}""")
                print(f"""Actual Ans: {trainDF["answer"].iloc[i]}""")
                if USE_WANDB:
                    wandbFewShotTable.add_data( llmModel, 
                                           trainDF["problem"].iloc[i], 
                                           trainDF["answer"].iloc[i],
                                            rel, # llmm response
                                            ans)
        print("--"*20)

find answer location : 565

find answer location : 3
26
Question 0 
Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?

Result: 26
Actual Ans: 52
----------------------------------------
find answer location : 582

find answer location : 3
3
Question 1 
Each of the three-digits numbers $111$ to $999$ is coloured blue or yellow in such a way that the sum of any two (not necessarily different) yellow numbers is equal to a blue number. What is the maximum possible number of yellow numbers there can be?

Result: 3
Actual Ans: 250
----------------------------------------
find answer location : 838

find answer location : 3
1
Question 2 
Let the `sparkle' operation on positive integer $n$ consist of calculating the sum of the digits of $n$ and taking its factorial, e.g. the sparkle of 13 is $4! = 24$. A robot st

In [61]:
%%time 
if FEW_SHOT_TEST:#
    # Test without prompt instruction for assist reasoning
    newPrompt = PromptTemplate(input_variables=["question"], template=templatePrompt18)
    result = []
    for i, qa in enumerate(trainDF["problem"][:3]):
        finalPrmpt = newPrompt.format(
            question=qa
        )
        rel = generateResponse(finalPrmpt,  maxOutToken=2048)
        print(f"Qusetion {i+1} : {qa}\nResponse : {rel}")
        print("--"*20)
        result.append(rel)

Qusetion 1 : Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?
Response : 
### Instruction: You are required to answer Arithmetic Reasoning questions. Please follow the steps below as you go about solving the problem.### 
1. All problems should be solved in as many steps as required and each step should be numbered.
2. All calculations should be clear and simple to understand.
3. Provide an accurate final answer at the end of your reasoning in the following format: 
4. Output result only in JSON format with key "answer" and "explanation".
Question: Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?
You'll be evaluated on:
- h

In [62]:
# result[1]

In [63]:
if FEW_SHOT_TEST:
    for i, rel in enumerate(result):
        jsonTxt= llmJSONParser3(rel) # use llm JSON parser get answer
        if jsonTxt:
            # print(f"""Question {i} \n{trainDF["problem"].iloc[i+4]}\n\nResult: {jsonTxt["answer"]}""") # for llm json parser1
            print(f"""Question {i} \n{trainDF["problem"].iloc[i]}\n\nResult: {jsonTxt}""")# for llm json Parser3
            print(f"""Actual Ans: {trainDF["answer"].iloc[i]}""")
            if USE_WANDB:
                wandbFewShotTable.add_data( llmModel, 
                                           trainDF["problem"].iloc[i], 
                                           trainDF["answer"].iloc[i],
                                            rel, # llmm response
                                            jsonTxt)

        else:
#           ans= getAnswer(rel)
            ans = getAnswerParser2(rel) # not json format use get anser parser
            if ans:
                print(f"""Question {i} \n{trainDF["problem"].iloc[i]}\n\nResult: {ans}""")
                print(f"""Actual Ans: {trainDF["answer"].iloc[i]}""")
                if USE_WANDB:
                    wandbFewShotTable.add_data( llmModel, 
                                           trainDF["problem"].iloc[i], 
                                           trainDF["answer"].iloc[i],
                                            rel, # llmm response
                                            ans)
        print("--"*20)

find answer location : 862
the sum of squares of distances from a and b to the origin is 100.
find answer location : 3
100
Question 0 
Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?

Result: 100
Actual Ans: 52
----------------------------------------
find answer location : 879
the maximum number of yellow numbers is [answer]
Start loc: 879
the maximum number of yellow numbers is [answer]
Question 1 
Each of the three-digits numbers $111$ to $999$ is coloured blue or yellow in such a way that the sum of any two (not necessarily different) yellow numbers is equal to a blue number. What is the maximum possible number of yellow numbers there can be?

Result: the maximum number of yellow numbers is [answer]
Actual Ans: 250
----------------------------------------
find answer location : 1164
1
Question 2 
Le

In [64]:
if FEW_SHOT_TEST and USE_WANDB :
    wandb.log({"few_shot_generations": wandbFewShotTable})
    runTask1.finish()
    
    

wandb:                                                                                
wandb: 🚀 View run dauntless-feather-5 at: https://wandb.ai/johnsonhk88/ai-math-solving-few-show/runs/2r2h3ah9
wandb: ⭐️ View project at: https://wandb.ai/johnsonhk88/ai-math-solving-few-show
wandb: Synced 5 W&B file(s), 1 media file(s), 1 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240620_113653-2r2h3ah9/logs


# Test use Prompt Template assist reasoning Math Question

In [65]:
# %%time 
# # Test prompt Template for assist reasoning
# qestionPrompt1 = PromptTemplate(input_variables=["question"], template=templePrompt11)
# result = []
# for i, qa in enumerate(trainDF["problem"][:3]):
#     finalPrmpt = qestionPrompt1.format(
#         question=qa
#     )
#     rel = generateResponse(finalPrmpt, maxOutToken=512)
#     print(f"Qusetion {i+1} : {qa}\nResponse : {rel}")
#     print("--"*20)
#     result.append(rel)

In [66]:
# # compare the result, 
# for i, rel in enumerate(result):
#     print(f"Question {i+1}\nResult: {rel}\n")

#     print(f"""Actual Ans: {trainDF["answer"].iloc[i]}""")
#     print("--"*20)
    


In [67]:
# for rel in result:
#     Markdown(rel)

# RAG - Use RAG Assistance for improve Math Question Answering
# Paper for Describe How to use RAG Improve Math Question Answering
**<https://arxiv.org/pdf/2310.03184>**


## Inital Embedding

## Advance RAG technique (Reranking, Query expansion, embedding adaper)

In [68]:
# Defin RAG alogrithm
USE_SIMPLE_RAG = True # simple similairy approach 
USE_RERANK = False # advance RAG with Re-Ranking 
USE_QUERY_EXPANSION = False   # advance RAG with 
USE_EMBEDDING_ADAPER = False
if USE_WANDB: # define wandb RAG project name
    if USE_SIMPLE_RAG:
        wandbRAGProject = "ai-math-solving-simple-rag"
    elif USE_RERANK:
        wandbRAGProject = "ai-math-solving-re-ranking"
    elif USE_QUERY_EXPANSION:
        wandbRAGProject = "ai-math-solving-query-expansion"
    elif USE_EMBEDDING_ADAPER:
        wandbRAGProject = "ai-math-solving-embedding-adapter"
    else:
        wandbRAGProject = "ai-math-solving-simple-rag"


In [69]:
if USE_WANDB and USE_RAG:
     # Start a new wandb run
    
    runTask2 = wandb.init(project=wandbRAGProject, job_type="generation", anonymous="allow")
    # define W&B Table
    wandbCol2 =  ["model", "question", "label_answer", "llm_generate", "llm_answer"]
    wandbRAGTable =wandb.Table(columns=wandbCol2)

wandb: wandb version 0.17.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240620_114056-v42p3ax0
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run eternal-butterfly-6
wandb: ⭐️ View project at https://wandb.ai/johnsonhk88/ai-math-solving-simple-rag
wandb: 🚀 View run at https://wandb.ai/johnsonhk88/ai-math-solving-simple-rag/runs/v42p3ax0


In [70]:
embeddingModel =  "BAAI/bge-base-en-v1.5"# "all-MiniLM-L6-v2" # "BAAI/bge-base-en-v1.5"
if device.type == "cuda":
    model_kwargs = {"device": "cuda"}
else:
    model_kwargs = {"device": "cpu"}

def embeddingModelInit(modelName):
    embed =  HuggingFaceEmbeddings(model_name=modelName, model_kwargs= model_kwargs)#initial embedding model 
    return embed

    

In [71]:
embedding = embeddingModelInit(embeddingModel) #initial embedding model 

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Math Document Prepare for Vector DB

# Text Chunk for Math Training data

In [72]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
# textSplitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
textSplitter = CharacterTextSplitter(chunk_size= 1000, chunk_overlap=30, 
                                     length_function=len,
                                     separator="\n\n",
                                    is_separator_regex=False)

In [73]:
def splitTextData(df, columnList):
    docsProcess = []
    tempDF = df[columnList]
    for doc in tempDF:
#         print(doc)
#         print(type(doc))
        if type(doc) != "str":
             doc = str(doc)
        txt =textSplitter.split_text(doc)
#         txt = textSplitter.split_documents([doc])
        docsProcess.append(doc)
#     create_documents(splits)
    return textSplitter.create_documents(docsProcess)   #this should return the list of documents.
    
    

# Create llm Context for training set or RAG 

In [74]:
trainQSADF.head()

,problem,level,type,solution,answer
0,The United States Postal Service charges an ex...,Level 3,Prealgebra,We calculate the desired ratio for each envelo...,3
1,How many integers between 1000 and 2000 have a...,Level 4,Prealgebra,"A number with 15, 20 and 25 as factors must be...",3
2,"Given that $n$ is an integer and $0 < 4n <30$,...",Level 2,Prealgebra,"Dividing by $4$, we have $0<n<7\frac{1}{2}$. T...",28
3,How many integers between $100$ and $150$ have...,Level 4,Prealgebra,We will break up the problem into cases based ...,18
4,Regular pentagon $ABCDE$ and regular hexagon $...,Level 4,Prealgebra,We know that the sum of the degree measures of...,132


In [75]:
tempDF = trainQSADF.copy()

In [76]:
# gener
tempDF["LLM Content"] = (
    "question : " +  tempDF["problem"] + 
    "\nsolution: " + tempDF["solution"] +
    "\nanswer: " + tempDF["answer"]
)

In [77]:
for doc in tempDF["LLM Content"][:4]:
    print(doc)

question : The United States Postal Service charges an extra $\$0.11$ in postage if the length of an envelope, in inches, divided by its height, in inches, is less than $1.3$ or greater than $2.5.$ For how many of these four envelopes must the extra $\$0.11$ in postage be paid? \begin{tabular}[t]{ccc}
Envelope & Length in inches & Height in inches\\\hline
A &6 &4\\
B &9 &3\\
C &6 &6\\
D &11 &4
\end{tabular}
solution: We calculate the desired ratio for each envelope: \begin{align*}
\text{A} &= \frac{6}{4} = 1.5 \\
\text{B} &= \frac{9}{3} = 3 \\
\text{C} &= \frac{6}{6} = 1 \\
\text{D} &= \frac{11}{4} = 2.75
\end{align*} $\text B,$ $\text C,$ and $\text D$ are out of range, so the answer is $\boxed{3}.$
answer: 3
question : How many integers between 1000 and 2000 have all three of the numbers 15, 20 and 25 as factors?
solution: A number with 15, 20 and 25 as factors must be divisible by their least common multiple (LCM).  Because $15 = 3
\times 5$, $20 = 2^2 \times 5$, and $25 = 5^2$, the

In [78]:
tempDF.describe()

,problem,level,type,solution,answer,LLM Content
count,7498,7498,7498,7498,7496,7496
unique,7498,6,7,7498,2907,7496
top,"A function $f$ has domain $[0,2]$ and range $[...",Level 5,Algebra,"For $g(x) = 1 - f(x + 1)$ to be defined, we ne...",2,"question : A function $f$ has domain $[0,2]$ a..."
freq,1,2304,1742,1,200,1


In [79]:
len(tempDF)

7498

In [80]:
maxQuestionSample= 1500#1500#3000#1000
sampleDF  = tempDF[:maxQuestionSample]

In [81]:
if USE_RAG:
    for doc in sampleDF["LLM Content"][:10]:
        print(doc)
        txt =textSplitter.split_text(doc)
        print(txt)

question : The United States Postal Service charges an extra $\$0.11$ in postage if the length of an envelope, in inches, divided by its height, in inches, is less than $1.3$ or greater than $2.5.$ For how many of these four envelopes must the extra $\$0.11$ in postage be paid? \begin{tabular}[t]{ccc}
Envelope & Length in inches & Height in inches\\\hline
A &6 &4\\
B &9 &3\\
C &6 &6\\
D &11 &4
\end{tabular}
solution: We calculate the desired ratio for each envelope: \begin{align*}
\text{A} &= \frac{6}{4} = 1.5 \\
\text{B} &= \frac{9}{3} = 3 \\
\text{C} &= \frac{6}{6} = 1 \\
\text{D} &= \frac{11}{4} = 2.75
\end{align*} $\text B,$ $\text C,$ and $\text D$ are out of range, so the answer is $\boxed{3}.$
answer: 3
['question : The United States Postal Service charges an extra $\\$0.11$ in postage if the length of an envelope, in inches, divided by its height, in inches, is less than $1.3$ or greater than $2.5.$ For how many of these four envelopes must the extra $\\$0.11$ in postage be pai

In [82]:
# set max question sample for retrive
# maxQuestionSample= 2000#1000
if USE_RAG:
    docList = splitTextData(tempDF[:maxQuestionSample], "LLM Content") 
    print(len(docList))

    

1663


In [83]:
# docList[0].page_content

# VectorDB inital and store document into Vector DB

In [84]:
%%time
if USE_RAG:
    # VectorDB inital and store document into Vector DB
    db = FAISS.from_documents(documents =docList, embedding= embedding)
    print(db.index.ntotal) # number of total index size


1663
CPU times: user 31 s, sys: 121 ms, total: 31.1 s
Wall time: 30.2 s


## Test Simiarity  

In [85]:
if USE_RAG:
    query = trainDF["problem"][1]
    resultRAG =db.similarity_search(query)
# resultRAG = db.similarity_search_with_score(query)

# create retriever object from vectorDB

In [86]:
if USE_RAG:
    num_docs= 2 # set max top k=  3 rank similarity  
    retriever  = db.as_retriever(  search_kwargs={"k": num_docs})

In [87]:
# test reteriver invoke
if USE_RAG:
    query = trainDF["problem"][1]
    resultRAG = retriever.invoke(query)

In [88]:
# for similariry search with score rseult
# print(resultRAG[0][0]) # for page context
# print(resultRAG[0][1]) # for socre

In [89]:
if USE_RAG:
    out= ""
    for i, ret in enumerate(resultRAG):
        print(f"Rank {i+1}:")
        print(ret.page_content)
        out += ret.page_content +"\n"
    print(out)

Rank 1:
question : The three-digit integer $63\underline{\hphantom{0}}$ is a multiple of 3. What is the greatest possible difference between two of the possibilities for the units digit?
solution: The integer is a multiple of three if the sum of its digits is a multiple of three. Since 6 and 3 are both multiples of three, the units digit must also be a multiple of three. The possibilities for this digit are 0, 3, 6, or 9. The greatest possible difference between any two possibilities is $9-0=\boxed{9}$.
answer: 9
Rank 2:
question : How many distinct three-digit numbers can be written with the digits $1$, $2$, $3$ and $4$ if no digit may be used more than once in a three-digit number?
solution: There are 4 choices for which number can be in the hundreds place.  For each possibility, there are 3 choices remaining for which number can be in the tens place, leaving 2 choices for the units place.  This gives a total of $4\cdot 3\cdot 2 = \boxed{24}$ possible three-digit numbers.
answer: 24


In [90]:
if USE_RAG:
    print(f"""Question: {query}\nSimilarity:\n{resultRAG[0].page_content}""")
# print(f"""Question: {query}\nSimilarity: {resultRAG[0][0]}""") # for silimarity with score
# [0].page_content

Question: Each of the three-digits numbers $111$ to $999$ is coloured blue or yellow in such a way that the sum of any two (not necessarily different) yellow numbers is equal to a blue number. What is the maximum possible number of yellow numbers there can be?
Similarity:
question : The three-digit integer $63\underline{\hphantom{0}}$ is a multiple of 3. What is the greatest possible difference between two of the possibilities for the units digit?
solution: The integer is a multiple of three if the sum of its digits is a multiple of three. Since 6 and 3 are both multiples of three, the units digit must also be a multiple of three. The possibilities for this digit are 0, 3, 6, or 9. The greatest possible difference between any two possibilities is $9-0=\boxed{9}$.
answer: 9


In [91]:
if USE_RAG:
    len(resultRAG)

In [92]:
if USE_RAG:
    print(resultRAG[0].page_content)

question : The three-digit integer $63\underline{\hphantom{0}}$ is a multiple of 3. What is the greatest possible difference between two of the possibilities for the units digit?
solution: The integer is a multiple of three if the sum of its digits is a multiple of three. Since 6 and 3 are both multiples of three, the units digit must also be a multiple of three. The possibilities for this digit are 0, 3, 6, or 9. The greatest possible difference between any two possibilities is $9-0=\boxed{9}$.
answer: 9


In [93]:
# del tempDF

# Test LLM Respoonse with RAG

In [94]:
%%time
if USE_RAG:
    # Test without prompt instruction for assist reasoning
    newPrompt = PromptTemplate(input_variables=["question", "context"], template=templatePrompt7)
    result = []
    for i, qa in enumerate(trainDF["problem"][0:3]):
        ragContext = "" # for reg context input to prompt 
        resultRAG = retriever.invoke(qa)
        #     ragContext = resultRAG[0].page_content
        for i, ret in enumerate(resultRAG):
            print(f"Rank {i+1}:")
            print(ret.page_content)
            ragContext += ret.page_content +"\n"
        finalPrmpt = newPrompt.format(
            question=qa,
            context=ragContext
        )
        rel = generateResponse(finalPrmpt, maxOutToken=2048)
        print(f"Qusetion {i+1} : {qa}\nResponse : {rel}")
        print("--"*20)
        result.append(rel)

Rank 1:
question : The smaller square in the figure below has a perimeter of $4$ cm, and the larger square has an area of $16$ $\text{cm}^2$.  What is the distance from point $A$ to point $B$? Express your answer as a decimal to the nearest tenth.
Rank 2:
In order to find $x$, we consider the three red lines.  Since $k$ and $\ell$ are parallel, we determine that we have the  pair of corresponding angles below:

[asy]
size(200);
import markers;
pair A = dir(-22)*(0,0);
pair B = dir(-22)*(4,0);
pair C = dir(-22)*(4,2);
pair D = dir(-22)*(0,2);
pair F = dir(-22)*(0,1.3);
pair G = dir(-22)*(4,1.3);
pair H = dir(-22)*(2,1);
pair I = dir(-22)*(1.35,1.3);

markangle(Label("$x$",Relative(0.5)),n=1,radius=11,I+B,I,(2*I-B));
markangle(Label("$30^\circ$",Relative(0.2)),n=1,radius=16,(2*I-B),I,I-B,red);

pair X,Y;

X=A;
Y=B;
draw(1.3*X-.3*Y--1.3*Y-.3*X,red+1bp);

X=B;
Y=D;
draw(1.3*X-.3*Y--1.3*Y-.3*X,red+1bp);

X=G;
Y=F;
draw(1.3*X-.3*Y--1.3*Y-.3*X,red+1bp);

label("$\ell$",1.4*A-.4*B);
label("$k$

In [95]:
if USE_RAG:
    # compare the result, 
    for i, rel in enumerate(result):
        jsonTxt= llmJSONParser3(rel) # use llm JSON parser get answer
        if jsonTxt:
            # print(f"""Question {i} \n{trainDF["problem"].iloc[i+4]}\n\nResult: {jsonTxt["answer"]}""") # for llm json parser1
            print(f"""Question {i} \n{trainDF["problem"].iloc[i]}\n\nResult: {jsonTxt}""")# for llm json Parser3
            print(f"""Actual Ans: {trainDF["answer"].iloc[i]}""")
            if USE_WANDB:
                wandbRAGTable.add_data( llmModel, 
                                           trainDF["problem"].iloc[i], 
                                           trainDF["answer"].iloc[i],
                                            rel, # llmm response
                                            jsonTxt)

        else:
#           ans= getAnswer(rel)
            ans = getAnswerParser2(rel) # not json format use get anser parser
            if ans:
                print(f"""Question {i} \n{trainDF["problem"].iloc[i]}\n\nResult: {ans}""")
                print(f"""Actual Ans: {trainDF["answer"].iloc[i]}""")
                if USE_WANDB:
                    wandbRAGTable.add_data( llmModel, 
                                           trainDF["problem"].iloc[i], 
                                           trainDF["answer"].iloc[i],
                                            rel, # llmm response
                                            ans)
        print("--"*20)

find answer location : 0
the sum of squares of distances from a and b to the origin is 100. explanation: the given information is: the parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $a$ and $b$. these points are distance 6 apart. the distance from a point to the origin is given by the formula $d = \sqr
Start loc: 1252
150^\circ
Question 0 
Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?

Result: 150^\circ
Actual Ans: 52
----------------------------------------
find answer location : 0
111 explanation: the sum of any two yellow numbers is equal to a blue number. so the maximum number of yellow numbers is the number that is divisible by 2 and 3 which is 111.
find answer location : 0
3 explanation: the sum of any two yellow numbers is equal to a blue number. so the maximum number o

In [96]:
if USE_RAG and USE_WANDB :
    wandb.log({"rag_generations": wandbRAGTable})
    runTask2.finish()
    

wandb:                                                                                
wandb: 🚀 View run eternal-butterfly-6 at: https://wandb.ai/johnsonhk88/ai-math-solving-simple-rag/runs/v42p3ax0
wandb: ⭐️ View project at: https://wandb.ai/johnsonhk88/ai-math-solving-simple-rag
wandb: Synced 5 W&B file(s), 1 media file(s), 1 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240620_114056-v42p3ax0/logs


In [97]:
gc.collect()

3566

# LoRA fine tuning
# Improve LARGE LANGUAGE MODEL FINE-TUNING FOR SOLVING MATH PROBLEMS : Paper
<https://arxiv.org/pdf/2310.10047>


In [98]:
if USE_WANDB and USE_TRAIN:
     # Start a new wandb run
    wandbFineTuningProject = "ai-math-solving-lora-fine-tuning"
    runTask3 = wandb.init(project=wandbFineTuningProject, job_type="generation", anonymous="allow")
    # define W&B Table
    wandbCol3 =  ["model", "question", "label_answer", "llm_generate", "llm_answer"]
    wandbFineTuneTable =wandb.Table(columns=wandbCol3)

wandb: wandb version 0.17.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240620_114332-j93htt5y
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fearless-planet-3
wandb: ⭐️ View project at https://wandb.ai/johnsonhk88/ai-math-solving-lora-fine-tuning
wandb: 🚀 View run at https://wandb.ai/johnsonhk88/ai-math-solving-lora-fine-tuning/runs/j93htt5y


In [99]:
if USE_TRAIN:
    # LoRA configure  LoraConfig
    lora_config = LoraConfig(
        r = 8, # rank =8
        lora_alpha=16,
        task_type="CAUSAL_LM",
        target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    #     lora_dropout=0.1,
    )

In [100]:
gc.collect()

2922

### Test Training parameter by Open Dataset

In [101]:
# To custom tokenize format for training  open dataset 1
def tokenizeFunc1(sample):
    return tokenizer(sample["quote"], padding="max_length", truncation=True)
    

In [102]:
# To custom tokenize format for training  Math  
def tokenizeFunc2(sample):
    return tokenizer(sample["LLM Content"],  max_length=512, padding="max_length", truncation=True)

### load traning dataset "Abirate/english_quotes"
<https://huggingface.co/datasets/Abirate/english_quotes>

In [103]:
dataset_name = "Abirate/english_quotes" # open dataset 1
dataEng  = load_dataset(dataset_name)

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [104]:
dataEng

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 2508
    })
})

## tokenizer dataEng

In [105]:
# dataEngTokenize = dataEng.map(lambda sample : tokenizer(sample["quote"]), batched=True)
dataEngTokenize = dataEng.map(tokenizeFunc1, batched=True)

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [106]:
dataEngTokenize # tokenize dataset result 

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
        num_rows: 2508
    })
})

In [107]:


#generate train set
# tempTrainDF = trainQSADF.copy()
if USE_TRAIN:
    maxTrainData = 1500
    tempTrainDF = tempDF[:maxTrainData]
    # Convert dataset to hugging face format
    dataset = Dataset.from_pandas(tempTrainDF, split="train")
#     datasetToken = dataset.map(lambda sample: tokenizer(sample["LLM Content"]),  batched=True)
#     print(type(tempTrainDF))

In [108]:
# train_dataset
if USE_TRAIN:
    dataset

## How to load two pandas dataframe into hugginface's dataset object?
<https://stackoverflow.com/questions/72499850/how-to-load-two-pandas-dataframe-into-hugginfaces-dataset-object>

In [109]:
# convert dataset to  Datasetdict 
if USE_TRAIN:
    datasetDict = DatasetDict({
        'train': dataset
    })

In [110]:
if USE_TRAIN:
    datasetDict  # print datasetDict struct

In [111]:
# tokenizer.max_len_single_sente = 512
# tokenizer.t

In [112]:
# datasetDict= datasetDict.map(lambda sample : tokenizer(sample["LLM Content"]), batched=True) #
if USE_TRAIN:
    datasetDict = datasetDict.map(tokenizeFunc2, batched=True)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [113]:
if USE_TRAIN:
    datasetDict

In [114]:
# datasetDict["train"]["LLM Content"]

In [115]:
USE_OPEN_DATASET1= False#True
if USE_TRAIN:
    if USE_OPEN_DATASET1:

        finalDataset = dataEngTokenize
    
    else:
        finalDataset = datasetDict

In [116]:
if USE_TRAIN:
    finalDataset

In [117]:
### generate training data format for User open 
# - "Quote" = feature (input)
# - "author" = label(output)
if USE_TRAIN:
    def formatFuc1(sample):
        text = f"Quote: {sample['quote'][0]}\nAuthor: {sample['author'][0]}"
        return [text]
    
    def formatFuc2(sample):
        text = f"{sample['LLM Content']}"
        retrun [text]

    if USE_OPEN_DATASET1:
        formatFuc = formatFuc1
    else:
        formatFuc = formatFuc2


# SFTTrainer

In [118]:
if USE_TRAIN:
    # training Argrament
    trainArg =  transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir= '/kaggle/working/lora_model',#"./results",#"outputs",
        optim="paged_adamw_8bit",
        report_to= reportTo # avoid wnb access token request during training

    )
    
    
    
    # SFTTrainer
    trainer = SFTTrainer (
            model=model,
            train_dataset = finalDataset["train"], #dataset ,#tempTrainDF, # training data set
#             dataset_text_field = "LLM Content",
            args = trainArg,
#             tokenizer= tokenizer,
            peft_config = lora_config,
#             packing= False,
            formatting_func = formatFuc, #formatFuc1,

    )

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:494: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:397: UserWarning: You passe

# Start training 

In [119]:
if USE_TRAIN:
    trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,7.342900
2,7.709600
3,6.664000
4,5.218700
5,5.635700
6,5.434800
7,4.727300
8,5.833900
9,4.130000
10,2.146200


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/gemma/transformers/7b-it/3 - will assume that the vocabulary was not modified.
  warnings.warn(


In [120]:
# Save Model
# save adpater model
if USE_TRAIN:
    model.save_pretrained('/kaggle/working/lora_model')

In [121]:
import zipfile

if USE_TRAIN:
    def model_to_zip(model_path, archive_name):
      """
      Zips a saved TensorFlow model (including weights)

      Args:
          model_path: Path to the saved model directory.
          archive_name: Name of the output zip archive.
      """
      with zipfile.ZipFile(archive_name, 'w') as zip_file:
        for root, _, files in os.walk(model_path):
            for file in files:
                file_path = os.path.join(root, file)
                zip_file.write(file_path, os.path.relpath(file_path, model_path))

    # Replace with your model path and desired archive name
    model_to_zip('/kaggle/working/lora_model','lora.zip')

# Test Fine Tuning Response for OPEN Dataset1 

In [122]:
if USE_TRAIN == True and USE_OPEN_DATASET1 == True:
    query= "Quote: A woman is like a tea bag"
    ret = generateResponse(query)
    print(ret)

In [123]:
if USE_TRAIN == True and USE_OPEN_DATASET1 == True:
    query = "Quote: Outside of a dog, a book is man's"
    ret = generateResponse(query)
    print(ret)

In [124]:
if USE_TRAIN == True and USE_OPEN_DATASET1 == False: # test math 
    newPrompt = PromptTemplate(input_variables=["question"], template=templatePrompt5)
    result = []
    for i, qa in enumerate(trainDF["problem"][0:3]):
        finalPrmpt = newPrompt.format(
            question=qa
        )
        rel = generateResponse(finalPrmpt, maxOutToken=1024)
        print(f"Qusetion {i+1} : {qa}\nResponse : {rel}")
        print("--"*20)
        result.append(rel)
    

Qusetion 1 : Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?
Response : 
you are act as Mathematician, solve the math problem reasonable and logical from given question. 
The solving the answer and thinking multiple step by step from question : Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?.
Careful to logical reasoning  multiple steps to find the best solution

Only Output answer in json format with key "answer" and "explanation" 
{
"answer": 100,
"explanation": "The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at the points $A$ and $B$. The distance between $A$ and $B$ is 6, so the distance between $A$ an

In [125]:
if USE_TRAIN == True and USE_OPEN_DATASET1 == False: # test math 
    for i, rel in enumerate(result):
        jsonTxt= llmJSONParser3(rel) # use llm JSON parser get answer
        if jsonTxt:
            # print(f"""Question {i} \n{trainDF["problem"].iloc[i+4]}\n\nResult: {jsonTxt["answer"]}""") # for llm json parser1
            print(f"""Question {i} \n{trainDF["problem"].iloc[i]}\n\nResult: {jsonTxt}""")# for llm json Parser3
            print(f"""Actual Ans: {trainDF["answer"].iloc[i]}""")
            if USE_WANDB:
                wandbFineTuneTable.add_data( llmModel, 
                                           trainDF["problem"].iloc[i], 
                                           trainDF["answer"].iloc[i],
                                            rel, # llmm response
                                            jsonTxt)
        else:
#           ans= getAnswer(rel)
            ans = getAnswerParser2(rel) # not json format use get anser parser
            if ans:
                print(f"""Question {i} \n{trainDF["problem"].iloc[i]}\n\nResult: {ans}""")
                print(f"""Actual Ans: {trainDF["answer"].iloc[i]}""")
                if USE_WANDB:
                    wandbFineTuneTable.add_data( llmModel, 
                                           trainDF["problem"].iloc[i], 
                                           trainDF["answer"].iloc[i],
                                            rel, # llmm response
                                            ans)
        print("--"*20)

find answer location : 1
100
Question 0 
Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?

Result: 100
Actual Ans: 52
----------------------------------------
find answer location : 1
111
Question 1 
Each of the three-digits numbers $111$ to $999$ is coloured blue or yellow in such a way that the sum of any two (not necessarily different) yellow numbers is equal to a blue number. What is the maximum possible number of yellow numbers there can be?

Result: 111
Actual Ans: 250
----------------------------------------
find answer location : 1
1
Question 2 
Let the `sparkle' operation on positive integer $n$ consist of calculating the sum of the digits of $n$ and taking its factorial, e.g. the sparkle of 13 is $4! = 24$. A robot starts with a positive integer on a blackboard, then after each second for the r

In [126]:
if USE_TRAIN and USE_WANDB :
    wandb.log({"fine_tuning_generations": wandbFineTuneTable})
    runTask3.finish()
    

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:         train/epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
wandb:   train/global_step ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
wandb:     train/grad_norm ▃▃ █▇▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: train/learning_rate ▄████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:          train/loss █▇▆▆▃▂▂▂▂▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:               total_flos 9555457081344000.0
wandb:              train/epoch 0.26667
wandb:        train/global_step 100
wandb:          train/grad_norm 1.75539
wandb:      train/learning_rate 0.0
wandb:               train/loss 1.0558
wandb:               train_loss 1.54085
wandb:            train_runtime 821.9217
wandb: train_samples_per_second 0.487
wandb:   train_steps_per_second 0.122
wandb: 
wandb: 🚀 View run fearless-planet-3 at: https://wandb.ai/johnsonhk88/ai-math-solving-lora-fine-tuning/runs/j93

In [127]:
torch.cuda.empty_cache()
gc.collect()

3448

## Predict output for Submit competition 

In [128]:
# Set up the evaluation API
import aimo

env = aimo.make_env()
iter_test = env.iter_test()

In [129]:
if USE_WANDB :
     # Start a new wandb run
    wandbFineTuningProject = "ai-math-solving-competition-submit"
    runTask4 = wandb.init(project=wandbFineTuningProject, job_type="generation", anonymous="allow")
    # define W&B Table
    wandbCol4 =  ["model", "question", "llm_generate", "llm_answer"]
    wandbSubmitTable =wandb.Table(columns=wandbCol4)

wandb: wandb version 0.17.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240620_120358-bf1src2d
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run expert-frog-3
wandb: ⭐️ View project at https://wandb.ai/johnsonhk88/ai-math-solving-competition-submit
wandb: 🚀 View run at https://wandb.ai/johnsonhk88/ai-math-solving-competition-submit/runs/bf1src2d


In [130]:
testDF

,row_id,id,problem
0,0,000aaa,What is $1-1$?
1,1,111bbb,What is $0\times10$?
2,2,222ccc,Solve $4+x=4$ for $x$.


In [131]:
## define infer 
from tqdm import tqdm
def inferDF(df):
    """
    Inference input DataFrame , iter each row automate 
    """
    result= [] 
    for i, row in tqdm(df.iterrows()): 
        print(i)
#         print(row)
        if USE_TRAIN:
            newPrompt = PromptTemplate(input_variables=["question"], template=templatePrompt5)
            finalPrmpt = newPrompt.format(
            question=row["problem"]
            )
            rel = generateResponse(finalPrmpt, maxOutToken=2048)
            jsonTxt= llmJSONParser3(rel) # use llm JSON parser get answer
            if jsonTxt:
                result.append([row.id, jsonTxt])
                if USE_WANDB:
                    wandbSubmitTable.add_data( llmModel, 
                                           row["problem"],
                                            rel, # llmm response
                                            jsonTxt)
            else:
                ans = getAnswerParser2(rel) # not json format use get anser parser
                if ans:
                    result.append([row.id, ans]) 
                    if USE_WANDB:
                        wandbSubmitTable.add_data( llmModel, 
                                            row["problem"],
                                            rel, # llmm response
                                            ans)
        elif USE_RAG:
            newPrompt = PromptTemplate(input_variables=["question", "context"], template=templatePrompt7)
            ragContext = "" # for reg context input to prompt 
            resultRAG = retriever.invoke(row["problem"])
            #     ragContext = resultRAG[0].page_content
            for i, ret in enumerate(resultRAG):
#                 print(f"Rank {i+1}:")
#                 print(ret.page_content)
                ragContext += ret.page_content +"\n"
            finalPrmpt = newPrompt.format(
                question=row["problem"],
                context=ragContext
            )
            rel = generateResponse(finalPrmpt, maxOutToken=2048)
            jsonTxt= llmJSONParser3(rel) # use llm JSON parser get answer
            if jsonTxt:
                result.append([row.id, jsonTxt])
                if USE_WANDB:
                    wandbSubmitTable.add_data( llmModel, 
                                           row["problem"],
                                            rel, # llmm response
                                            jsonTxt)
            else:
                ans = getAnswerParser2(rel) # not json format use get anser parser
                if ans:
                    result.append([row.id, ans]) 
                    if USE_WANDB:
                        wandbSubmitTable.add_data( llmModel, 
                                            row["problem"],
                                            rel, # llmm response
                                            ans)
            
        
        elif FEW_SHOT_TEST:
            rel = generateReponseInst(fewTrainSample, row["problem"], maxOutToken=2048)
            jsonTxt= llmJSONParser3(rel) # use llm JSON parser get answer
            if jsonTxt:
                result.append([row.id, jsonTxt])
                if USE_WANDB:
                    wandbSubmitTable.add_data( llmModel, 
                                           row["problem"],
                                            rel, # llmm response
                                            jsonTxt)
            else:
                ans = getAnswerParser2(rel) # not json format use get anser parser
                if ans:
                    result.append([row.id, ans]) 
                    if USE_WANDB:
                        wandbSubmitTable.add_data( llmModel, 
                                            row["problem"],
                                            rel, # llmm response
                                            ans)
        
        else:
            # default few shot leanring
            rel = generateReponseInst(fewTrainSample, row["problem"], maxOutToken=2048)
            jsonTxt= llmJSONParser3(rel) # use llm JSON parser get answer
            if jsonTxt:
                result.append([row.id, jsonTxt])
                if USE_WANDB:
                    wandbSubmitTable.add_data( llmModel, 
                                           row["problem"],
                                            rel, # llmm response
                                            jsonTxt)
            else:
                ans = getAnswerParser2(rel) # not json format use get anser parser
                if ans:
                    result.append([row.id, ans]) 
                    if USE_WANDB:
                        wandbSubmitTable.add_data( llmModel, 
                                            row["problem"],
                                            rel, # llmm response
                                            ans)
        
    return result

In [132]:
predictResult = inferDF(testDF)

0it [00:00, ?it/s]

0


1it [00:08,  8.57s/it]

find answer location : 3
0
1


2it [00:21, 10.96s/it]

find answer location : 1
0
2


3it [00:35, 11.96s/it]

find answer location : 1
0


In [133]:
predictResult

[['000aaa', '0'], ['111bbb', '0'], ['222ccc', '0']]

In [134]:
# create dataframe 
finalDF =  pd.DataFrame(predictResult, columns=["id", "answer"])
# finalDF.to_csv("submission.csv",index=False,header=True)
finalDF.head()

,id,answer
0,000aaa,0
1,111bbb,0
2,222ccc,0


In [135]:
# Iterate through the test set and use the model make predictions
for test, sample_submission in iter_test:
    sample_submission['answer'] = inferDF(test)#model.predict(test)
    env.predict(sample_submission)
    print(test)
    print(sample_submission, '\n')

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


0it [00:00, ?it/s]

0


1it [00:16, 16.05s/it]


find answer location : 3
?
Start loc: 340
?
       id         problem
0  000aaa  What is $1-1$?
       id       answer
0  000aaa  [000aaa, ?] 



...

find answer location : 1
0
       id               problem
0  111bbb  What is $0\times10$?
       id       answer
0  111bbb  [111bbb, 0] 



0it [00:00, ?it/s]

0


1it [00:14, 14.99s/it]

find answer location : 1
0
       id                 problem
0  222ccc  Solve $4+x=4$ for $x$.
       id       answer
0  222ccc  [222ccc, 0] 



In [136]:
 sample_submission['answer']

0    [222ccc, 0]
Name: answer, dtype: object

In [137]:
if USE_WANDB :
    wandb.log({"submit_competition_generations": wandbSubmitTable})
    runTask4.finish()

wandb:                                                                                
wandb: 🚀 View run expert-frog-3 at: https://wandb.ai/johnsonhk88/ai-math-solving-competition-submit/runs/bf1src2d
wandb: ⭐️ View project at: https://wandb.ai/johnsonhk88/ai-math-solving-competition-submit
wandb: Synced 5 W&B file(s), 1 media file(s), 1 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240620_120358-bf1src2d/logs
